범위: 핸즈온 7장 앙상블 학습과 랜덤 포레스트 p.245-257

##7.1 투표 기반 분류기

여러 분류기(로지스틱 회귀, SVM, 랜덤 포레스트, KNN 등)를 학습시켜 각각의 예측을 모아 가장 많이 선택된 클래스를 최종 예측으로 결정 → 직접 투표(hard voting) 분류기.

개별 분류기의 정확도가 낮더라도 충분히 많고 다양한 분류기를 결합하면 더 강력한 학습기(강한 학습기)가 될 수 있음.


[분류기의 정확도와 앙상블]

정확도 51%인 분류기 1,000개를 모아 다수결로 예측 → 이론상 약 75% 정확도 달성 가능.

단, 모든 분류기가 완벽히 독립적이고 오류가 상관관계가 없다는 가정 하에서 가능.

실제로는 같은 데이터로 학습하기 때문에 오류가 비슷해져 정확도가 떨어질 수 있음.

[TIP]

앙상블은 서로 독립적인 분류기일 때 효과 극대화.

다양한 분류기를 확보하는 방법: 서로 다른 알고리즘으로 학습시키기.

이렇게 하면 서로 다른 종류의 오류를 만들어 낼 가능성이 높아지고, 모델의 정확도가 향상됨.

##7.2 배깅과 페이스팅

배깅(Bagging): 훈련 세트에서 중복 허용하여 샘플링(bootstrap).

페이스팅(Pasting): 중복 비허용 샘플링.

같은 훈련 샘플을 여러 예측기에 사용 가능.

배깅/페이스팅의 특징

모든 예측기의 결과를 모아 최종 예측 → 분류기는 최빈값(mode), 회귀는 평균 사용.

원본 데이터로 단일 학습한 것보다 편향은 비슷하지만 분산은 줄어듦.

예측기들이 병렬 학습 가능 → CPU/서버 병렬 처리로 확장성 높음.

###7.2.1 사이킷런의 배깅/페이스팅

사이킷런은 BaggingClassifier / BaggingRegressor 제공.

bootstrap=True → 배깅, False → 페이스팅.

n_jobs → CPU 코어 수 지정.

예: 결정트리 500개 앙상블 학습 코드 제시.

결과: 단일 결정트리보다 더 작은 분산과 더 안정된 예측.

###7.2.2 oob 평가

배깅에서 평균적으로 훈련 샘플의 **63%**만 선택 → 나머지 37%는 oob 샘플.

oob 샘플은 별도 검증 세트 없이 평가에 사용 가능.

oob_score=True 설정 시 자동으로 oob 평가.

예시: BaggingClassifier 결과 → oob 정확도 약 90.1%, 테스트 세트 정확도 91.2%.

oob_decision_function_으로 각 클래스에 속할 확률 확인 가능.

##7.3 랜덤 패치와 랜덤 서브스페이스

BaggingClassifier는 특성 샘플링 지원.

매개변수: max_features, bootstrap_features.

max_samples, bootstrap과 동일 방식이지만 입력 특성에 대한 샘플링.

각 예측기는 무작위로 선택한 일부 특성으로 학습.

고차원 데이터셋(예: 이미지)에 유용.

[방식 구분]

랜덤 패치(random patches):

훈련 샘플 + 특성 모두 샘플링.

(bootstrap=False, max_samples=1.0) + (bootstrap_features=True 또는 max_features<1.0).

랜덤 서브스페이스(random subspaces):

특성만 샘플링.

[효과]

특성 샘플링은 다양한 예측기를 만들어 편향은 늘리고, 분산은 낮춤.

##7.4 랜덤 포레스트

랜덤 포레스트(Random Forest) = 결정 트리 기반의 배깅(또는 페이스팅) 앙상블.

RandomForestClassifier / RandomForestRegressor 제공.

BaggingClassifier + DecisionTreeClassifier 조합을 최적화해 사용 편리.

트리 노드를 분할할 때 전체 특성이 아니라 무작위 일부 특성 후보 중 최적 선택 → 다양성↑, 분산↓.

###7.4.1 엑스트라 트리 (Extremely Randomized Trees, Extra-Trees)

랜덤 포레스트와 유사하지만 특성을 무작위로 분할 → 극단적으로 무작위 트리 생성.

편향은 늘지만 분산은 크게 줄어듦.

최적 분할 계산이 필요 없어 더 빠름.

구현: ExtraTreesClassifier, ExtraTreesRegressor.

###7.4.2 특성 중요도

랜덤 포레스트는 **특성 중요도(feature_importances_)**를 쉽게 측정.

기준: 각 노드의 불순도 감소량(가중치 평균).

중요도의 합은 1이 되도록 정규화됨.

예: iris 데이터 → 꽃잎 길이(44%), 꽃잎 너비(42%)가 중요, 꽃받침 길이(11%), 너비(2%)는 덜 중요.

MNIST 데이터에서도 각 픽셀의 중요도를 시각화 가능.

##7.5 부스팅

부스팅(boosting): 약한 학습기를 여러 개 연결해 강한 학습기를 만드는 앙상블 방법.

아이디어: 앞의 모델을 보완하면서 순차적으로 학습기를 학습.

여러 방법 중 가장 인기 있는 것:

에이다부스트(AdaBoost, adaptive boosting)

그레이디언트 부스팅(gradient boosting)

In [10]:
import warnings
warnings.filterwarnings('ignore')

# import package
import numpy as np
import os

# 5장에서의 moons dataset 불러오기
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X,y = make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf=LogisticRegression()
rnd_clf=RandomForestClassifier()
svm_clf=SVC()

voting_clf=VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [13]:


from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred=clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8
RandomForestClassifier 0.95
SVC 0.95
VotingClassifier 0.95


In [3]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(x_train, y_train)
    y_pred=clf.predict(x_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.95
RandomForestClassifier 0.95
SVC 1.0
VotingClassifier 1.0


In [15]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf=BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=50, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred=bag_clf.predict(X_test)


bag_clf=BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_


0.9375

In [17]:
from sklearn.metrics import accuracy_score
y_pred=bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.95

In [18]:
bag_clf.oob_decision_function_

array([[0.99428571, 0.00571429],
       [0.        , 1.        ],
       [0.83783784, 0.16216216],
       [0.34946237, 0.65053763],
       [0.93678161, 0.06321839],
       [0.        , 1.        ],
       [0.72580645, 0.27419355],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.01775148, 0.98224852],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.96875   , 0.03125   ],
       [0.99465241, 0.00534759],
       [0.        , 1.        ],
       [0.90285714, 0.09714286],
       [0.985     , 0.015     ],
       [0.87096774, 0.12903226],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.00549451, 0.99450549],
       [0.00555556, 0.99444444],
       [0.05759162, 0.94240838],
       [0.30978261, 0.69021739],
       [0.00578035, 0.99421965],
       [0.43786982, 0.56213018],
       [0.90857143, 0.09142857],
       [0.        , 1.        ],
       [0.03684211, 0.96315789],
       [0.

In [19]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf=RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf=rnd_clf.predict(X_test)

In [20]:
bag_clf=BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)

In [21]:
from sklearn.datasets import load_iris
iris=load_iris()
rnd_clf=RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.09771990296286917
sepal width (cm) 0.022353061096653533
petal length (cm) 0.44116915728692446
petal width (cm) 0.43875787865355287
